<a href="https://colab.research.google.com/github/sanjibb/spark_example/blob/master/Spark_Knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook shows an example of pySpark Knn Model. This has been integrated with sklearn.

In [0]:
%pyspark
from pyspark import SparkConf
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
import pandas as pd
import sklearn
#knn.fit(X_train,y_train)

print('The pandas version is {}.'.format(pd.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

url = 'https://ibm.box.com/shared/static/teo1syd2k4wwyfswg1slfhdn7u5a0b9l.txt'
col_names = (['Label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13',
             'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
             'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'])
data_df = pd.read_table(url,sep='\t', header=None, names=col_names)


Bring dataframe from sql

In [0]:
%pyspark

from pyspark.sql import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#provinfo1 = sqlContext.sql("""select asrpt_type_cd,subj_geo_state_cd,asrpt_cst_amt  from fps2_models.asrpt where asrpt_cst_amt is not null limit 100""")
provinfo2 = sqlContext.sql("""select case when asrpt_cst_amt > 100000 then 1 else 0 end label from fps2_models.asrpt where asrpt_cst_amt is not null """)
#provinfo1.show(10)
#provinfo2.show(10)

Use string indexer to make categorical variables to numeric

In [0]:
from pyspark.ml.feature import IndexToString, StringIndexer

df = sqlContext.sql("""select asrpt_id as id,subj_geo_state_cd as category,asrpt_cst_amt  from  fps2_models.asrpt where asrpt_cst_amt is not null""")

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = indexer.fit(df)
indexed = model.transform(df)

print("Transformed string column '%s' to indexed column '%s'"
      % (indexer.getInputCol(), indexer.getOutputCol()))
indexed.show()

#print("StringIndexer will store labels in output column metadata\n")

#converter = IndexToString(inputCol="categoryIndex", outputCol="originalCategory")
#converted = converter.transform(indexed)

#print("Transformed indexed column '%s' back to original string column '%s' using "
#      "labels in metadata" % (converter.getInputCol(), converter.getOutputCol()))
#converted.select("id", "categoryIndex", "originalCategory").show()

feature = indexed.select("id","asrpt_cst_amt","categoryIndex")


Use Encoder for categorical transformation to numeric

In [0]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["id", "categoryIndex"],
                                 outputCols=["idx", "categoryVec2"])
model = encoder.fit(indexed)
encoded = model.transform(indexed)
encoded.show()

Make list with feature and label column

In [0]:
X = [list(row) for row in feature.collect()]
y = [list(row) for row in provinfo2.collect()]

print(len(X))
print(len(y))

Bring train_test_split function

In [0]:
%pyspark
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=0)



Print number of rows for Train and Test Dataset

In [0]:
print('Number of rows in X_train is {}.'.format(len(X_train)))
print('Number of rows in X_test is {}.'.format(len(X_test)))
print('Number of rows in y_train is {}.'.format(len(y_train)))
print('Number of rows in y_test is {}.'.format(len(y_test)))

Call Knn Classifier

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
# K=5 for KNN (the n_neighbors parameter)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print('Accuracy score using k-nearest neighbor = {}'.format(accuracy_score(y_test,y_pred)))
print('AUC percentage using k-nearest neighbor = {}'.format(roc_auc_score(y_test,y_pred)))